### Week 10 - GDAL subprocess
Revisitng the subprocess module - whilst working on compiling final code, I decided that typing long lines of text into the command line wasn't going to work well and high chance of error. Fortunately was succesful in revisitng subprocess and getting it to work!

In [14]:
from osgeo import gdal
import numpy
import os
import rasterio as rio
import subprocess

In [36]:
#file directories 
be_raster = r"H:\Englishman\2021\DEMs\Bare_earth\EGM_BE_wgs84_1m.tif" 
snowon_raster = r"H:\Englishman\2021\DEMs\co-registered_STM\EGM_P01_STM_coreg.tif"
outfile = r"H:\Englishman\2021\DEMs\testing.tif"

#my gdal install - located in my anaconda environment
gdalcalc= r"C:\Anaconda\envs\geog650\Scripts\gdal_calc.py"

In [ ]:
#function to subtract rasters to create snow depth raster
def gdal_calc(bareearth_raster, snowon_raster, gdalcalc, snowdepth_raster):
    #Gdal calc command to call from the terminal, need to call the directory of where the gdalcalc script is installed. 
    cmd_calc = 'python {} -A {} -B {} --outfile={} --calc="A-B"'.format(gdalcalc,snowon_raster, bareearth_raster, snowdepth_raster)
    subprocess.run(cmd_calc, shell=True) #should exit with code 0 if succesful

In [ ]:
#function to first warp the "snowon_raster" to the bare earth if different extents
def gdal_warp(bareearth_raster, snowon_raster, be_read):
    #create the file name by adding warp to the snowon raster file name 
    warp_directory = str(snowon_directory)[:-4] + "_warp.tif"
    #Find the bounding box for the bare earth raster
    min_x=be_read.bounds[0]
    max_x=be_read.bounds[2]
    min_y=be_read.bounds[1]
    max_y=be_read.bounds[3]
    cmd_warp = 'gdalwarp -te {} {} {} {} {} {}'.format(min_x, min_y, max_x, max_y, snowon_raster, warp_directory)
    subprocess.run(cmd_warp, shell=True) #should exit with code 0 if succesful
    #return the output warped snow_on raster
    return warp_directory

In [ ]:
#check if the be and snow on are the same size by reading the rasters:
be_read = rio.open(be_directory)
snowon_read = rio.open(snowon_directory)
if be.shape != snow_on.shape:
    #call function to warp raster
    warp = gdal_warp(bareearth_raster, snowon_raster, be_read)
    #call function to subtract rasters using the warped raster as the snowon raster
    gdal_calc(bareearth_raster, warp, gdalcalc, snowdepth_raster)

### Below code 2 code blocks work!

In [49]:
#check if the be and snow on are the same size by reading the rasters:
be_read = rio.open(be_directory)
snowon_read = rio.open(snowon_directory)
if be.shape != snow_on.shape:
    #create new file name - adding warp extension to the snow-on raster
    warp_directory = str(snowon_directory)[:-4] + "_warp.tif"
    #Find the bounding box for the bare earth raster
    min_x=be.bounds[0]
    max_x=be.bounds[2]
    min_y=be.bounds[1]
    max_y=be.bounds[3]
    #gdalwarp -te min_x min_y max_x max_y inraster.tif outraster.tif
    #This one works!
    cmd_warp = 'gdalwarp -te {} {} {} {} {} {}'.format(min_x, min_y, max_x, max_y, snowon_directory, warp_directory)
    subprocess.run(cmd_warp, shell=True) #should exit with code 0 if succesful
    cmd_calc = 'python {} -A {} -B {} --outfile={} --calc="A-B"'.format(gdalcalc,warp_directory, be_directory, outfile)
    #subtract the bare earth from the snow on raster
    subprocess.run(cmd_calc, shell=True) #should exit with code 0 if succesful
    

bare earth and snow on raster are different sizes
H:\Englishman\2021\DEMs\co-registered_STM\EGM_P01_STM_coreg_warp.tif


In [51]:
#This one works!
cmd_warp = 'gdalwarp -te {} {} {} {} {} {}'.format(min_x, min_y, max_x, max_y, snowon_directory, warp_directory)
subprocess.run(cmd_warp, shell=True)

CompletedProcess(args='gdalwarp -te 380836.0 5449162.0 392000.0 5465596.0 H:\\Englishman\\2021\\DEMs\\co-registered_STM\\EGM_P01_STM_coreg.tif H:\\Englishman\\2021\\DEMs\\co-registered_STM\\EGM_P01_STM_coreg_warp.tif', returncode=0)

In [53]:
cmd_calc = 'python {} -A {} -B {} --outfile={} --calc="A-B"'.format(gdalcalc,warp_directory, be_directory, outfile)
#subtract the bare earth from the snow on raster
subprocess.run(cmd_calc, shell=True) #should exit with code 0 if succesful

CompletedProcess(args='python C:\\Anaconda\\envs\\geog650\\Scripts\\gdal_calc.py -A H:\\Englishman\\2021\\DEMs\\co-registered_STM\\EGM_P01_STM_coreg_warp.tif -B H:\\Englishman\\2021\\DEMs\\Bare_earth\\EGM_BE_wgs84_1m.tif --outfile=H:\\Englishman\\2021\\DEMs\\testing.tif --calc="A-B"', returncode=0)

In [48]:
cmd_warp = 'gdalwarp -te {} {} {} {} {} {}'.format(min_x, min_y, max_x, max_y, be_directory, warp_directory)
subprocess.run(cmd_warp, shell=True)

gdalwarp -te 380836.000 5449162.000 392000.000 5465596.000 H:\Englishman\2021\DEMs\co-registered_STM\EGM_P05_STM_coreg.tif H:\Englishman\2021\DEMs\co-registered_STM\EGM_P05_warp.tif

CompletedProcess(args='gdalwarp -te 381077.0 5449480.0 392000.0 5465525.0 H:\\Englishman\\2021\\DEMs\\Bare_earth\\EGM_BE_wgs84_1m.tif H:\\Englishman\\2021\\DEMs\\co-registered_STM\\EGM_P01_STM_coreg_warp.tif', returncode=1)

In [9]:
#check if the be and snow on are the same size:
be = rio.open(be)
snowon = rio.open(snowon)
if be.shape != snow_on.shape:
    print("bare earth and snow on raster are different sizes")
    file_output = str(snowon)[:-4] + "_warp.tif"
    print(file_output)
    
    #snowon = gdal.Warp()
    #warp, and then call the subprocess to calc

else:
    cmd = 'python {} -A {} -B {} --outfile={} --calc="A-B"'.format(gdalcalc,warp_directory, be_directory, outfile)
    #subtract the bare earth from the snow on raster
    subprocess.run(cmd, shell=True)
    

    meta = snow_on.meta
    #proceed to gdal calc function
    be = be.read(1)
    snow_on = snow_on.read(1)
    meta = snow_on.meta 
    sd = snow_on - be
    

AttributeError: 'numpy.ndarray' object has no attribute 'meta'

In [19]:
cmd_string = 'python C:\Anaconda\envs\geog650\Scripts\gdal_calc.py -A H:\\Englishman\\2021\\DEMs\\co-registered_STM\\EGM_P01_warp.tif -B H:\\Englishman\\2021\\DEMs\\bare_earth\\EGM_BE_wgs84_1m.tif --outfile=H:\\Englishman\\2021\\DEMs\\TEST_subprocess_2.tif --calc="A-B"'
subprocess.run(cmd_string, shell=True)

CompletedProcess(args='python C:\\Anaconda\\envs\\geog650\\Scripts\\gdal_calc.py -A H:\\Englishman\\2021\\DEMs\\co-registered_STM\\EGM_P01_warp.tif -B H:\\Englishman\\2021\\DEMs\\bare_earth\\EGM_BE_wgs84_1m.tif --outfile=H:\\Englishman\\2021\\DEMs\\TEST_subprocess_2.tif --calc="A-B"', returncode=0)

In [22]:
cmd_string_test = 'python ' + str(gdalcalc) + ' -A ' + str(snowon) + ' -B ' + str(be) + ' --outfile=' + str(outfile) + ' --calc="A-B"'
display(cmd_string_test)

'python C:\\Anaconda\\envs\\geog650\\Scripts\\gdal_calc.py -A H:\\Cruickshank\\2021\\DEMs\\Coregistered_STM\\CRU_P01_coregSTM.tif -B H:\\Cruickshank\\2021\\DEMs\\Bare_earth --outfile=H:\\Cruickshank\\2021\\DEMs\\testing_subprocess_cmd.tif --calc="A-B"'

In [33]:
#Subs in variables to the command string
cmd_string_test_2 = 'python {} -A {} -B {} --outfile={} --calc="A-B"'.format(gdalcalc,snowon, be, outfile)
#call subprocess command - activating the shell to run, ensure the environment required is active
subprocess.run(cmd_string_test_2, shell=True)

CompletedProcess(args='python C:\\Anaconda\\envs\\geog650\\Scripts\\gdal_calc.py -A H:\\Cruickshank\\2021\\DEMs\\Coregistered_STM\\CRU_P01_coregSTM.tif -B H:\\Cruickshank\\2021\\DEMs\\Bare_earth\\CRU_be.tif --outfile=H:\\Cruickshank\\2021\\DEMs\\test.tif --calc="A-B"', returncode=0)

In [ ]:
sd_out = "CRU_P01_SD.tif"
#output the tif file
with rio.open(sd_out, "w", **meta) as dst:
    dst.write(sd, 1)

In [ ]:
#Assign the bounding box for the to 
min_x = be.bounds[0]
max_x=be.bounds[2]
min_y=be.bounds[1]
max_y=be.bounds[3]
print(min_x, max_x, min_y, max_y)